### Imports

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import PyPDF2
import numpy as np
import re

2014-2016 need pages 7-9, beore 2014 pages 6-8

### Acquire NCAA attendance files from the NCAA website using BeautifulSoup

In [4]:
url = 'http://www.ncaa.org/championships/statistics/ncaa-football-attendance'

response = requests.get(url)
page = response.text
soup = BeautifulSoup(page,"html5lib")

In [5]:
attendance_files = []

pdf_search = soup.find(class_ = "field-item even").find_all('a')
#print(pdf_search)
for pdf in pdf_search:
    attendance_files.append(pdf.get('href'))
for link in attendance_files:
    print(link)

http://fs.ncaa.org/Docs/stats/football_records/Attendance/2016.pdf
http://fs.ncaa.org/Docs/stats/football_records/Attendance/2015.pdf
http://fs.ncaa.org/Docs/stats/football_records/Attendance/2014.pdf
http://fs.ncaa.org/Docs/stats/football_records/Attendance/2013Release.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2012.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2011.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2010.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2009.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2008.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2007.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2006.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2005.pdf
http://web1.ncaa.org/web_files/stats/football_records/Attendance/2004.pdf
http://web1.ncaa.org/web_files/stats/football_records/Atten

### Save PDF files locally

In [ ]:
path = '/Users/murdock/Documents/PDF_files/'
for file in attendance_files:
    url = file
    response = requests.get(url)
    with open(path + file.split('/')[-1], 'wb') as f:
        f.write(response.content)

### Make lists (manually) for all FCA and FBS teams with _ characters to eventually do a reshaping into a df

In [6]:
list_of_fcs_teams = ['Alabama A&M', 'Alabama St.', 'Albany (NY)', 'Alcorn', 'Alcorn St.', 'Ark.-Pine Bluff', 'Austin Peay', 
                      'Bethune-Cookman', 'Brown', 'Bryant', 'Bucknell', 'Butler', 'Cal Poly', 'Campbell',
                      'Central Ark', 'Central Conn. St.', 'Charleston So.', 'Chattanooga', 'Coastal Caro.', 'Colgate', 'Columbia',
                      'Cornell', 'Dartmouth', 'Davidson', 'Dayton', 'Delaware', 'Delaware St.', 'Drake', 'Duquesne',
                      'Eastern Ill.', 'Eastern Ky.', 'Eastern Wash.', 'Elon', 'ETSU', 'Florida A&M', 'Fordham',
                      'Furman', 'Gardner-Webb', 'Georgetown', 'Grambling', 'Hampton', 'Harvard', 'Holy Cross',
                      'Houston Baptist', 'Howard', 'Idaho St.', 'Illinois St.', 'Indiana St.', 'Jackson St.',
                      'Jacksonville', 'Jacksonville St.', 'James Madison', 'Kennesaw St.', 'Lafayette',
                      'Lamar University', 'Lehigh', 'Liberty', 'Maine', 'Marist', 'McNeese', 'Mcneese St.', 'Mercer',
                      'Mississippi Val.', 'Missouri St.', 'Monmouth', 'Montana', 'Montana St.', 'Morehead St.',
                      'Morgan St.', 'Murray St.', 'N.C. A&T', 'N.C. Central', 'New Hampshire', 'Nicholls St.',
                      'Norfolk St.', 'North Dakota', 'North Dakota St.', 'Northern Ariz.', 'Northern Colo.',
                      'Northwestern St.', 'Penn', 'Portland St.', 'Prairie View', 'Presbyterian', 'Princeton',
                      'Rhode Island', 'Richmond', 'Robert Morris', 'Sacramento St.', 'Sacred Heart',
                      'Saint Francis (PA)', 'St. Francis (PA)', 'Sam Houston St.', 'Samford', 'San Diego', 'Savannah St.', 'South Alabama', 'SFA',
                      'South Carolina St.', 'South Dakota', 'South Dakota St.', 'Southeast Mo. St.',
                      'Southeastern La.', 'Southern Ill.', 'Southern U.', 'Southern Utah', 'Stephen F. Austin', 'Stetson', 'Stony Brook',
                      'Tennessee St.', 'Tennessee Tech', 'Texas Southern', 'The Citadel', 'Towson', 'UC Davis',
                      'UNI', 'UT Martin', 'Valparaiso', 'Villanova', 'VMI', 'Wagner', 'Weber St.', 'Western Caro.',
                      'Western Ill.', 'William & Mary', 'Wofford', 'Yale', 'Youngstown St.']

replacement_fcs_names = ['Alabama_A_M', 'Alabama_St', 'Albany_NY', 'Alcorn_St', 'Alcorn_St', 'Ark_Pine_Bluff', 'Austin_Peay',
                         'Bethune_Cookman', 'Brown', 'Bryant', 'Bucknell', 'Butler', 'Cal_Poly', 'Campbell',
                         'Central_Ark', 'Central_Conn_St', 'Charleston_So', 'Chattanooga', 'Coastal_Caro', 'Colgate', 'Columbia',
                         'Cornell', 'Dartmouth', 'Davidson', 'Dayton', 'Delaware', 'Delaware_St', 'Drake',
                         'Duquesne', 'Eastern_Ill', 'Eastern_Ky', 'Eastern_Wash', 'Elon', 'ETSU', 'Florida_A_M',
                         'Fordham', 'Furman', 'Gardner_Webb', 'Georgetown', 'Grambling', 'Hampton', 'Harvard',
                         'Holy_Cross', 'Houston_Baptist', 'Howard', 'Idaho_St', 'Illinois_St', 'Indiana_St',
                         'Jackson_St', 'Jacksonville', 'Jacksonville_St', 'James_Madison', 'Kennesaw_St',
                         'Lafayette', 'Lamar_University', 'Lehigh', 'Liberty', 'Maine', 'Marist', 'McNeese', 'Mcneese_St',
                         'Mercer', 'Mississippi_Val', 'Missouri_St', 'Monmouth', 'Montana', 'Montana_St',
                         'Morehead_St', 'Morgan_St', 'Murray_St', 'N_C_A_T', 'N_C_Central', 'New_Hampshire',
                         'Nicholls_St', 'Norfolk_St', 'North_Dakota', 'North_Dakota_St', 'Northern_Ariz',
                         'Northern_Colo', 'Northwestern_St', 'Penn', 'Portland_St', 'Prairie_View', 'Presbyterian', 'Princeton',
                         'Rhode_Island', 'Richmond', 'Robert_Morris', 'Sacramento_St', 'Sacred_Heart',
                         'Saint_Francis_PA', 'Saint_Francis_PA', 'Sam_Houston_St', 'Samford', 'San_Diego', 'Savannah_St', 'South_Alabama', 'SFA',
                         'South_Carolina_St', 'South_Dakota', 'South_Dakota_St', 'Southeast_Mo_St',
                         'Southeastern_La', 'Southern_Ill', 'Southern_U', 'Southern_Utah', 'Stephen_F_Austin', 'Stetson', 
                         'Stony_Brook', 'Tennessee_St', 'Tennessee_Tech', 'Texas_Southern', 'The_Citadel',
                         'Towson', 'UC_Davis', 'UNI', 'UT_Martin', 'Valparaiso', 'Villanova', 'VMI', 'Wagner',
                         'Weber_St', 'Western_Caro', 'Western_Ill', 'William_Mary', 'Wofford', 'Yale',
                         'Youngstown_St']

In [7]:
list_of_fbs_teams = ['Air Force', 'Akron', 'Alabama', 'Appalachian St.', 'Arizona', 'Arizona St.', 'Arkansas', 'Arkansas St.',
                     'Army West Point', 'Auburn', 'Ball St.', 'Baylor', 'Boise St.', 'Boston College',
                     'Bowling Green', 'Buffalo', 'BYU', 'California', 'Central Mich.', 'Charlotte', 'Cincinnati',
                     'Clemson', 'Colorado', 'Colorado St.', 'Duke', 'East Carolina', 'Eastern Mich.', 'FIU',
                     'Fla. Atlantic', 'Florida', 'Florida St.', 'Fresno St.', 'Ga. Southern', 'Georgia',
                     'Georgia St.', 'Georgia Tech', 'Hawaii', 'Houston', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
                     'Iowa St.', 'Kansas', 'Kansas St.', 'Kent St.', 'Kentucky', 'La.-Monroe', 'Louisiana',
                     'Louisiana Tech', 'Louisville', 'LSU', 'Marshall', 'Maryland', 'Massachusetts', 'Memphis',
                     'Miami (FL)', 'Miami (OH)', 'Michigan', 'Michigan St.', 'Middle Tenn.', 'Minnesota',
                     'Mississippi St.', 'Missouri', 'Navy', 'Nebraska', 'Nevada', 'New Mexico', 'New Mexico St.',
                     'North Carolina', 'North Carolina St.', 'North Texas', 'Northern Ill.', 'Northwestern',
                     'Notre Dame', 'Ohio', 'Ohio St.', 'Oklahoma', 'Oklahoma St.', 'Old Dominion', 'Ole Miss',
                     'Oregon', 'Oregon St.', 'Penn St.', 'Pittsburgh', 'Purdue', 'Rice', 'Rutgers', 'San Diego St.',
                     'San Jose St.', 'SMU', 'South Ala.', 'South Carolina', 'South Fla.', 'Southern California',
                     'Southern Miss.', 'Stanford', 'Syracuse', 'TCU', 'Temple', 'Tennessee', 'Texas', 'Texas A&M',
                     'Texas St.', 'Texas Tech', 'Toledo', 'Troy', 'Tulane', 'Tulsa', 'UCF', 'UCLA', 'UConn',
                     'UNLV', 'Utah', 'Utah St.', 'UTEP', 'UTSA', 'Vanderbilt', 'Virginia', 'Virginia Tech',
                     'Wake Forest', 'Washington', 'Washington St.', 'West Virginia', 'Western Ky.', 'Western Mich.',
                     'Wisconsin', 'Wyoming']

replacement_fbs_names = ['Air_Force', 'Akron', 'Alabama', 'Appalachian_St', 'Arizona', 'Arizona_St', 'Arkansas', 'Arkansas_St',
                         'Army_West_Point', 'Auburn', 'Ball_St', 'Baylor', 'Boise_St', 'Boston_College',
                         'Bowling_Green', 'Buffalo', 'BYU', 'California', 'Central_Mich', 'Charlotte', 'Cincinnati',
                         'Clemson', 'Colorado', 'Colorado_St', 'Duke', 'East_Carolina', 'Eastern_Mich', 'FIU',
                         'Fla_Atlantic', 'Florida', 'Florida_St', 'Fresno_St', 'Ga_Southern', 'Georgia',
                         'Georgia_St', 'Georgia_Tech', 'Hawaii', 'Houston', 'Idaho', 'Illinois', 'Indiana', 'Iowa',
                         'Iowa_St', 'Kansas', 'Kansas_St', 'Kent_St', 'Kentucky', 'La_Monroe', 'Louisiana',
                         'Louisiana_Tech', 'Louisville', 'LSU', 'Marshall', 'Maryland', 'Massachusetts', 'Memphis',
                         'Miami_FL', 'Miami_OH', 'Michigan', 'Michigan_St', 'Middle_Tenn', 'Minnesota',
                         'Mississippi_St', 'Missouri', 'Navy', 'Nebraska', 'Nevada', 'New_Mexico', 'New_Mexico_St',
                         'North_Carolina', 'North_Carolina_St', 'North_Texas', 'Northern_Ill', 'Northwestern',
                         'Notre_Dame', 'Ohio', 'Ohio_St.', 'Oklahoma', 'Oklahoma_St', 'Old_Dominion', 'Ole_Miss',
                         'Oregon', 'Oregon_St', 'Penn_St', 'Pittsburgh', 'Purdue', 'Rice', 'Rutgers', 'San_Diego_St',
                         'San_Jose_St', 'SMU', 'South_Ala', 'South_Carolina', 'South_Fla', 'Southern_California',
                         'Southern_Miss', 'Stanford', 'Syracuse', 'TCU', 'Temple', 'Tennessee', 'Texas', 'Texas_A_M',
                         'Texas_St', 'Texas_Tech', 'Toledo', 'Troy', 'Tulane', 'Tulsa', 'UCF', 'UCLA', 'UConn',
                         'UNLV', 'Utah', 'Utah_St', 'UTEP', 'UTSA', 'Vanderbilt', 'Virginia', 'Virginia_Tech',
                         'Wake_Forest', 'Washington', 'Washington_St', 'West_Virginia', 'Western_Ky', 'Western_Mich',
                         'Wisconsin', 'Wyoming']

In [8]:
fbs = list(zip(list_of_fbs_teams, replacement_fbs_names))
fcs = list(zip(list_of_fcs_teams, replacement_fcs_names))

Combine FCS and FBS teams into one list

In [9]:
all_teams_sorted = sorted(fcs + fbs, key=lambda t: len(t[0]), reverse=True)
all_teams_sorted

[('Southern California', 'Southern_California'),
 ('Saint Francis (PA)', 'Saint_Francis_PA'),
 ('South Carolina St.', 'South_Carolina_St'),
 ('North Carolina St.', 'North_Carolina_St'),
 ('Central Conn. St.', 'Central_Conn_St'),
 ('Southeast Mo. St.', 'Southeast_Mo_St'),
 ('Stephen F. Austin', 'Stephen_F_Austin'),
 ('Jacksonville St.', 'Jacksonville_St'),
 ('Lamar University', 'Lamar_University'),
 ('Mississippi Val.', 'Mississippi_Val'),
 ('North Dakota St.', 'North_Dakota_St'),
 ('Northwestern St.', 'Northwestern_St'),
 ('St. Francis (PA)', 'Saint_Francis_PA'),
 ('South Dakota St.', 'South_Dakota_St'),
 ('Southeastern La.', 'Southeastern_La'),
 ('Ark.-Pine Bluff', 'Ark_Pine_Bluff'),
 ('Bethune-Cookman', 'Bethune_Cookman'),
 ('Houston Baptist', 'Houston_Baptist'),
 ('Sam Houston St.', 'Sam_Houston_St'),
 ('Appalachian St.', 'Appalachian_St'),
 ('Army West Point', 'Army_West_Point'),
 ('Mississippi St.', 'Mississippi_St'),
 ('Charleston So.', 'Charleston_So'),
 ('Northern Ariz.', 'Nort

### Look at one year of data to get workflow optimized

In [10]:
path_1 = '/Users/murdock/Documents/PDF_files/2016.pdf'
pdfFileObj_1 = open(path_1, 'rb')
pdfReader_1 = PyPDF2.PdfFileReader(pdfFileObj_1)

In [11]:
pageObj_1 = pdfReader_1.getPage(6)
sp = pageObj_1.extractText()
pageObj_2 = pdfReader_1.getPage(7)
sp2 = pageObj_2.extractText()
pageObj_3 = pdfReader_1.getPage(8)
sp3 = pageObj_3.extractText()

In [12]:
combined_pages = sp + ' ' + sp2 + ' ' + sp3
combined_pages

"201\n6 NCAA DIVISION I \nFBS \nTEAM\n-BY-TEAM \nHOME \nATTENDANCE\n  School\n G Attendance\n Average\n Air Force\n 6 177,519 29,587 Akron\n 6 62,021 10,337 Alabama\n 7 712,747 101,821 Appalachian St.\n 6 156,916 26,153 Arizona\n 7 338,017 48,288 Arizona St.\n 6 286,417 47,736 Arkansas\n 7 487,067 69,581 Arkansas St.\n 6 136,200 22,700 Army West Point\n 5 163,267 32,653 Auburn\n 8 695,498 86,937 Ball St.\n 5 38,946 7,789 Baylor\n 6 275,029 45,838 Boise St.\n 6 205,640 34,273 Boston College\n 6 192,942 32,157 Bowling Green\n 6 90,838 15,140 Buffalo\n 6 104,957 17,493 BYU\n 6 351,413 58,569 California\n 6 279,769 46,628 Central Mich.\n 6 104,447 17,408 Charlotte\n 6 85,154 14,192 Cincinnati\n 7 235,095 33,585 Clemson\n 7 566,787 80,970 Colorado\n 6 279,652 46,609 Colorado St.\n 6 165,598 27,600 Duke\n 6 179,369 29,895 East Carolina\n 6 264,680 44,113 Eastern M\nich.\n 6 106,064 17,677 FIU\n 7 117,526 16,789 Fla. Atlantic\n 6 60,437 10,073 Florida\n 5 439,229 87,846 Florida St.\n 6 460,80

##### Test function to replace school name from text with school name from provided list

In [13]:
cleaned_string = combined_pages.replace('\n','').lower()
def replace_text(text, list_of_names):
    for name in list_of_names:
        if name[0].lower() in text:
            text = text.replace(name[0].lower(), name[1].lower())
    return text

pages = replace_text(cleaned_string, all_teams_sorted)
pages

"2016 ncaa division i fbs team-by-team home attendance  school g attendance average air_force 6 177,519 29,587 akron 6 62,021 10,337 alabama 7 712,747 101,821 appalachian_st 6 156,916 26,153 arizona 7 338,017 48,288 arizona_st 6 286,417 47,736 arkansas 7 487,067 69,581 arkansas_st 6 136,200 22,700 army_west_point 5 163,267 32,653 auburn 8 695,498 86,937 ball_st 5 38,946 7,789 baylor 6 275,029 45,838 boise_st 6 205,640 34,273 boston_college 6 192,942 32,157 bowling_green 6 90,838 15,140 buffalo 6 104,957 17,493 byu 6 351,413 58,569 california 6 279,769 46,628 central_mich 6 104,447 17,408 charlotte 6 85,154 14,192 cincinnati 7 235,095 33,585 clemson 7 566,787 80,970 colorado 6 279,652 46,609 colorado_st 6 165,598 27,600 duke 6 179,369 29,895 east_carolina 6 264,680 44,113 eastern_mich 6 106,064 17,677 fiu 7 117,526 16,789 fla_atlantic 6 60,437 10,073 florida 5 439,229 87,846 florida_st 6 460,801 76,800 fresno_st 6 152,960 25,493 ga_southern 5 104,095 20,819 georgia 6 556,476 92,746 geor

##### Workflow for cleaning text from pdf

In [14]:
left_text = pages.partition("  2016 ncaa division ii team-by-team football")[0]
left_text2 = re.sub('2016 ncaa division i fbs team-by-team home attendance  ', '', left_text)
left_text3 = re.sub('2016 ncaa division i fcs team-by-team home attendance  ', '', left_text2)
left_text4 = re.sub('  ', ' ', left_text3)
left_text5 = re.sub('school g attendance average ', '', left_text4)
print(left_text5)

air_force 6 177,519 29,587 akron 6 62,021 10,337 alabama 7 712,747 101,821 appalachian_st 6 156,916 26,153 arizona 7 338,017 48,288 arizona_st 6 286,417 47,736 arkansas 7 487,067 69,581 arkansas_st 6 136,200 22,700 army_west_point 5 163,267 32,653 auburn 8 695,498 86,937 ball_st 5 38,946 7,789 baylor 6 275,029 45,838 boise_st 6 205,640 34,273 boston_college 6 192,942 32,157 bowling_green 6 90,838 15,140 buffalo 6 104,957 17,493 byu 6 351,413 58,569 california 6 279,769 46,628 central_mich 6 104,447 17,408 charlotte 6 85,154 14,192 cincinnati 7 235,095 33,585 clemson 7 566,787 80,970 colorado 6 279,652 46,609 colorado_st 6 165,598 27,600 duke 6 179,369 29,895 east_carolina 6 264,680 44,113 eastern_mich 6 106,064 17,677 fiu 7 117,526 16,789 fla_atlantic 6 60,437 10,073 florida 5 439,229 87,846 florida_st 6 460,801 76,800 fresno_st 6 152,960 25,493 ga_southern 5 104,095 20,819 georgia 6 556,476 92,746 georgia_st 6 90,617 15,103 georgia_tech 7 332,518 47,503 hawaii 7 170,299 24,328 houston

In [15]:

left_text2 = re.sub('2016 ncaa division i fbs team-by-team home attendance  ', '', left_text)
left_text3 = re.sub('2016 ncaa division i fcs team-by-team home attendance  ', '', left_text2)
left_text4 = re.sub('  ', ' ', left_text3)
left_text5 = re.sub('school g attendance average ', '', left_text4)
print(left_text5)


air_force 6 177,519 29,587 akron 6 62,021 10,337 alabama 7 712,747 101,821 appalachian_st 6 156,916 26,153 arizona 7 338,017 48,288 arizona_st 6 286,417 47,736 arkansas 7 487,067 69,581 arkansas_st 6 136,200 22,700 army_west_point 5 163,267 32,653 auburn 8 695,498 86,937 ball_st 5 38,946 7,789 baylor 6 275,029 45,838 boise_st 6 205,640 34,273 boston_college 6 192,942 32,157 bowling_green 6 90,838 15,140 buffalo 6 104,957 17,493 byu 6 351,413 58,569 california 6 279,769 46,628 central_mich 6 104,447 17,408 charlotte 6 85,154 14,192 cincinnati 7 235,095 33,585 clemson 7 566,787 80,970 colorado 6 279,652 46,609 colorado_st 6 165,598 27,600 duke 6 179,369 29,895 east_carolina 6 264,680 44,113 eastern_mich 6 106,064 17,677 fiu 7 117,526 16,789 fla_atlantic 6 60,437 10,073 florida 5 439,229 87,846 florida_st 6 460,801 76,800 fresno_st 6 152,960 25,493 ga_southern 5 104,095 20,819 georgia 6 556,476 92,746 georgia_st 6 90,617 15,103 georgia_tech 7 332,518 47,503 hawaii 7 170,299 24,328 houston

In [16]:
words = left_text5.split(' ')
word_array = np.array(words).reshape(-1, 4)
word_array

array([['air_force', '6', '177,519', '29,587'],
       ['akron', '6', '62,021', '10,337'],
       ['alabama', '7', '712,747', '101,821'],
       ['appalachian_st', '6', '156,916', '26,153'],
       ['arizona', '7', '338,017', '48,288'],
       ['arizona_st', '6', '286,417', '47,736'],
       ['arkansas', '7', '487,067', '69,581'],
       ['arkansas_st', '6', '136,200', '22,700'],
       ['army_west_point', '5', '163,267', '32,653'],
       ['auburn', '8', '695,498', '86,937'],
       ['ball_st', '5', '38,946', '7,789'],
       ['baylor', '6', '275,029', '45,838'],
       ['boise_st', '6', '205,640', '34,273'],
       ['boston_college', '6', '192,942', '32,157'],
       ['bowling_green', '6', '90,838', '15,140'],
       ['buffalo', '6', '104,957', '17,493'],
       ['byu', '6', '351,413', '58,569'],
       ['california', '6', '279,769', '46,628'],
       ['central_mich', '6', '104,447', '17,408'],
       ['charlotte', '6', '85,154', '14,192'],
       ['cincinnati', '7', '235,095', '33,5

In [17]:
dataf = pd.DataFrame(word_array)

In [18]:
years = dataf.head()

In [19]:
years = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
path_1 = '/Users/murdock/Documents/PDF_files/2016.pdf'
pdfFileObj_1 = open(path_1, 'rb')
pdfReader_1 = PyPDF2.PdfFileReader(pdfFileObj_1)

In [20]:
pageObj_1 = pdfReader_1.getPage(6)
sp = pageObj_1.extractText()
pageObj_2 = pdfReader_1.getPage(7)
sp2 = pageObj_2.extractText()
pageObj_3 = pdfReader_1.getPage(8)
sp3 = pageObj_3.extractText()

### Make final functions to parse all PDF files

In [21]:
def replace_text(text, list_of_names):
    """
    Finds and replaces all team neames in a text with names that have underscores
    
    INPUTS:
    text - a string of text
    list_of_names - the list of all schools and the names that should be replaced
    
    RETURNS:
    The original text with the school names replaced to names with underscores
    """
    for name in list_of_names:
        if name[0].lower() in text:
            text = text.replace(name[0].lower(), name[1].lower())
    return text

In [22]:
def read_pdf_files():
    """
    Parses all pdf files from the desired years to obtain attendance records for all schools
    
    INPUTS:
    none as the years and all information are inside the function.
    
    RETURNS:
    a dataframe with every division I team's attendances for the given years
    """
    df = pd.DataFrame()
    year = []
    years = ['2009', '2010', '2011', '2012', '2013Release', '2014', '2015', '2016']
    base = '/Users/murdock/Documents/PDF_files/'
    for year in years:
        #print(year)
        path = base + year + '.pdf'
        pdfFileObj_1 = open(path, 'rb')
        pdfReader_1 = PyPDF2.PdfFileReader(pdfFileObj_1)
        if years.index(year) < 5:
            pageObj_1 = pdfReader_1.getPage(5)
            sp = pageObj_1.extractText()
            pageObj_2 = pdfReader_1.getPage(6)
            sp2 = pageObj_2.extractText()
            pageObj_3 = pdfReader_1.getPage(7)
            sp3 = pageObj_3.extractText()
            if year == '2013Release':
                s1 = '2013 ncaa division i fbs team-by-team attendance  '
                s2 = '2013 ncaa division i fcs team-by-team attendance  '
            else:
                s1 = '{} ncaa division i fbs team-by-team attendance  '.format(year)
                s2 = '{} ncaa division i fcs team-by-team attendance  '.format(year)
        else:
            pageObj_1 = pdfReader_1.getPage(6)
            sp = pageObj_1.extractText()
            pageObj_2 = pdfReader_1.getPage(7)
            sp2 = pageObj_2.extractText()
            pageObj_3 = pdfReader_1.getPage(8)
            sp3 = pageObj_3.extractText()
            s1 = '{} ncaa division i fbs team-by-team home attendance  '.format(year)
            s2 = '{} ncaa division i fcs team-by-team home attendance  '.format(year)
        combined_pages = sp + ' ' + sp2 + ' ' + sp3
        cleaned_string = combined_pages.replace('\n','').lower()
        pages = replace_text(cleaned_string, all_teams_sorted)
        if year == '2013Release':
            left_text = pages.partition(" 2013 ncaa division ii team-by-team football")[0]
        elif year == '2016':
            left_text = pages.partition("  {} ncaa division ii team-by-team football".format(year))[0]
        elif years.index(year) > 4:
            left_text = pages.partition(" {} ncaa division ii team-by-team football".format(year))[0]
        else:
            left_text = pages.partition("  {} ncaa division ii team-by-team football".format(year))[0]
        left_text2 = re.sub(s1, '', left_text)
        left_text3 = re.sub(s2, '', left_text2)
        left_text4 = re.sub('  ', ' ', left_text3)
        left_text5 = re.sub('school g attendance average ', '', left_text4)
        words = left_text5.split(' ')
        word_array = np.array(words).reshape(-1, 4)
        #print(word_array.shape)
        word_df = pd.DataFrame(word_array)
        df = pd.concat([df, word_df], ignore_index=True)
    return df

### making a dataframe of school info and pickling

In [23]:
df = read_pdf_files()

In [24]:
team_years = np.repeat(('2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'), (238, 237, 241, 249, 247, 245, 250, 250))
team_years

array(['2009', '2009', '2009', ..., '2016', '2016', '2016'],
      dtype='<U4')

In [25]:
df.columns = ['School', 'Home_Games', 'Attendance', 'Average']

df.head()

,School,Home_Games,Attendance,Average
0,air_force,6,"213,937","35,656"
1,akron,6,"104,293","17,382"
2,alabama,7,"644,084","92,012"
3,arizona,6,"315,330","52,555"
4,arizona_st,7,"339,890","48,556"


In [26]:
df['Year'] = team_years
df.head()

,School,Home_Games,Attendance,Average,Year
0,air_force,6,"213,937","35,656",2009
1,akron,6,"104,293","17,382",2009
2,alabama,7,"644,084","92,012",2009
3,arizona,6,"315,330","52,555",2009
4,arizona_st,7,"339,890","48,556",2009


In [ ]:
df_path = '/Users/murdock/Documents/PDF_files/'
df.to_pickle(df_path + 'attendance_df.pkl')